In [2]:
# generate_dataset.py (終極版 - 緊密跟隨 & 全員多據點)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_vehicle_behavior_dataset():
    """
    生成一個為期兩個月，包含多種使用角色化車牌的複雜行為模式軌跡資料集。
    此版本實現了更緊密的跟隨行為，並為所有毒販團夥成員增加了不固定的停留點。
    """
    print("--- 開始生成資料集 (終極版 - 緊密跟隨 & 全員多據點) ---")

    # --- 步驟 1: 定義虛擬世界 (攝影機網路) ---
    # ... (此部分程式碼維持不變) ...
    camera_data = {
        '攝影機': [
            'A01', 'A02', 'A03', 'A04', 'A05', # 區域 A: 主要住宅區
            'B01', 'B02', 'B03', 'B04', 'B05', # 區域 B: 辦公/商業區
            'C01', 'C02', 'C03',              # 區域 C: 次要住宅區/安全屋
            'D01', 'D02', 'D03',              # 區域 D: 休閒娛樂區/過夜點
            'S01', 'S02',              # 區域 S: 可疑停留點
            'P01', 'P02', 'P03', 'P04',        # 區域 P: 交易點
            'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07' # 交通幹道
        ],
        '攝影機名稱': [
            '楊梅區-A住宅區-巷口', '楊梅區-A幹道-路口1', '楊梅區-A幹道-路口2', '楊梅區-A超市前', '楊梅區-交流道A',
            '中壢區-B園區-入口', '中壢區-B園區-內部', '中壢區-B商業大道-1', '中壢區-B商業大道-2', '中壢區-交流道B',
            '平鎮區-C住宅區-巷口', '平鎮區-C大學路', '平鎮區-C車站',
            '龍潭區-D風景區-入口', '龍潭區-D湖畔道路', '龍潭區-D市區',
            '龜山區-S工業區-路口', '龜山區-S倉庫前',
            '八德區-P公園-角落', '大溪區-P停車場-後方', '觀音區-P快速道路下', '新屋區-P濱海涼亭',
            '交通幹道-AS段', '交通幹道-AB段', '交通幹道-BC段', '交通幹道-CD段', '交通幹道-AD段', '交通幹道-BS段', '交通幹道-CP段'
        ],
        '單位': (['楊梅派出所']*5 + ['中壢派出所']*5 + ['平鎮派出所']*3 + 
               ['龍潭派出所']*3 + ['龜山派出所']*2 + ['八德分局']*4 + ['交通大隊']*7)
    }
    total_cameras = len(camera_data['攝影機'])
    camera_data['緯度'] = np.linspace(24.8, 25.1, total_cameras)
    camera_data['經度'] = np.linspace(121.1, 121.4, total_cameras)
    cameras_df = pd.DataFrame(camera_data)
    print(f"1. 攝影機網路建立完成，共 {total_cameras} 支攝影機。")

    paths = {
        "A_to_B": ['A01', 'A05', 'T02', 'B05', 'B01', 'B02'], "B_to_A": ['B02', 'B01', 'B05', 'T02', 'A05', 'A01'],
        "A_to_S": ['A01', 'A05', 'T01', 'S01'], "S_to_B": ['S01', 'T06', 'B05', 'B01', 'B02'],
        "B_to_D": ['B02', 'B03', 'T03', 'C02', 'T04', 'D01'],
        "C_to_P": ['C01', 'T07', 'P01', 'P02'], "P_to_A": ['P02', 'P04', 'T05', 'A02', 'A01'],
        "P_to_C": ['P01','T07','C02','C01'], "P_to_D": ['P04','T04','D02','D03'],
        "B_internal_1": ['B01', 'B03', 'B04'], "B_internal_2": ['B04', 'B05', 'B01'],
    }

    # --- 步驟 2: 定義車輛角色與生成邏輯 ---
    plates = {
        "normal":     [f"NOR-{1001+i}" for i in range(5)],
        "dealer":     [f"DEL-8888"],
        "accomplice": [f"ACC-0001", "ACC-0002"],
        "uncertain":  [f"UNC-1001", "UNC-2001", "UNC-3001"],
        "sparse":     [f"SPA-7777", "SPA-8888"],
        "territorial":[f"TER-9999"]
    }
    all_sightings = []

    def generate_trip(plate, path, start_time, time_variance_factor=1.0):
        # ... (此函式不變) ...
        current_time = start_time
        for cam_id in path:
            delay = random.uniform(8, 12) * time_variance_factor
            current_time += timedelta(minutes=delay)
            cam_info = cameras_df[cameras_df['攝影機'] == cam_id].iloc[0]
            all_sightings.append({'車牌': plate, '日期': current_time.strftime('%Y-%m-%d'), '時間': current_time.strftime('%H:%M:%S'), **cam_info.to_dict()})
        return current_time

    def generate_stay(plate, location_cam_id, start_time, duration_hours):
        # ... (此函式不變) ...
        end_time = start_time + timedelta(hours=duration_hours)
        cam_info = cameras_df[cameras_df['攝影機'] == location_cam_id].iloc[0]
        all_sightings.append({'車牌': plate, '日期': start_time.strftime('%Y-%m-%d'), '時間': start_time.strftime('%H:%M:%S'), **cam_info.to_dict()})
        all_sightings.append({'車牌': plate, '日期': end_time.strftime('%Y-%m-%d'), '時間': end_time.strftime('%H:%M:%S'), **cam_info.to_dict()})
        return end_time
    
    print("2. 車輛角色與生成邏輯定義完成。")

    # --- 步驟 3: 執行兩個月的「劇本」---
    print("3. 開始執行兩個月的每日劇本...")
    start_date = datetime(2025, 8, 1)
    end_date = datetime(2025, 9, 30)
    
    # 為毒販團夥設定共享的事件日期和據點
    dealer_run_dates = [d.date() for d in pd.date_range(start_date, end_date) if random.random() < 0.4]
    accomplice_bases = ['A04', 'C03', 'D02']
    accomplice1_current_base = random.choice(accomplice_bases)
    accomplice2_current_base = random.choice(accomplice_bases)

    current_date = start_date
    while current_date <= end_date:
        day_of_week = current_date.weekday()

        # --- 正常上班族劇本 ---
        for plate in plates["normal"]:
            if day_of_week < 5 and random.random() < 0.95:
                # ... (此劇本邏輯不變) ...
                variance = 1.8 if random.random() < 0.1 else 1.0
                start_morning = current_date.replace(hour=8, minute=random.randint(0, 45))
                end_morning = generate_trip(plate, paths['A_to_B'], start_morning, time_variance_factor=variance)
                generate_stay(plate, 'B02', end_morning, duration_hours=random.uniform(8, 9))
                start_evening = end_morning + timedelta(hours=random.uniform(8,9.5))
                generate_trip(plate, paths['B_to_A'], start_evening, time_variance_factor=variance)
        
        # --- 毒販與跟班協同劇本 (緊密跟隨 & 全員多據點) ---
        dealer_plate = plates["dealer"][0]
        accomplice1_plate, accomplice2_plate = plates["accomplice"]

        if current_date.date() in dealer_run_dates:
            start_run = current_date.replace(hour=22, minute=random.randint(0, 59))
            
            # 透過速度差異（時間變異因子）模擬緊密跟隨
            # 探路車，速度快 5%
            generate_trip(accomplice1_plate, paths['C_to_P'], start_run, time_variance_factor=0.95)
            # 主車，正常速度
            current_time = generate_trip(dealer_plate, paths['C_to_P'], start_run, time_variance_factor=1.0)
            # 殿後車，速度慢 5%
            generate_trip(accomplice2_plate, paths['C_to_P'], start_run, time_variance_factor=1.05)
            
            # 主車在交易點停留後返回 A 區過夜點
            current_time += timedelta(minutes=random.randint(5,15))
            end_time = generate_trip(dealer_plate, paths['P_to_A'], current_time)
            generate_stay(dealer_plate, 'A01', end_time, random.uniform(6,10))
            
            # 跟班們各自返回一個隨機的、不固定的據點
            accomplice1_current_base = random.choice(accomplice_bases)
            accomplice2_current_base = random.choice(accomplice_bases)
            generate_stay(accomplice1_plate, accomplice1_current_base, start_run + timedelta(hours=2), random.uniform(7,10))
            generate_stay(accomplice2_plate, accomplice2_current_base, start_run + timedelta(hours=2.5), random.uniform(7,10))
        else:
            # 在沒有任務的日子裡，跟班們也會隨機移動並更換據點
            if random.random() < 0.5:
                 start_move = current_date.replace(hour=random.randint(10,14))
                 accomplice1_current_base = random.choice(accomplice_bases)
                 end_move = generate_trip(accomplice1_plate, random.choice([paths['A_to_B'], paths['B_to_D']]), start_move)
                 generate_stay(accomplice1_plate, accomplice1_current_base, end_move, random.uniform(8,12))
        
        # --- 地盤型車輛劇本 ---
        # ... (此劇本邏輯不變) ...
        territorial_plate = plates["territorial"][0]
        if day_of_week < 5 and random.random() < 0.9:
             start_work = current_date.replace(hour=9, minute=random.randint(0, 20))
             current_time = generate_stay(territorial_plate, 'B01', start_work, random.uniform(0.2,0.5))
             for _ in range(random.randint(4, 8)):
                 path_key = random.choice(["B_internal_1", "B_internal_2"])
                 current_time = generate_trip(territorial_plate, paths[path_key], current_time)
                 current_time = generate_stay(territorial_plate, random.choice(['B03', 'B04', 'B05']), current_time, random.uniform(0.1, 0.3))
        
        # --- 資料不足車輛 ---
        # ... (此劇本邏輯不變) ...
        for plate in plates["sparse"]:
             if random.random() < 0.05:
                 start_time = current_date.replace(hour=random.randint(9,18))
                 generate_trip(plate, random.choice(list(paths.values())), start_time)

        current_date += timedelta(days=1)
    print("劇本執行完畢。")

    # --- 步驟 4: 格式化並儲存檔案 ---
    if all_sightings:
        print("4. 開始格式化並儲存最終資料...")
        final_df = pd.DataFrame(all_sightings)
        final_df['攝影機_int'] = final_df['攝影機'].apply(lambda x: abs(hash(x)) % (10**9) + 2000000000)
        final_df = final_df.drop(columns=['攝影機'])
        final_df = final_df.rename(columns={'攝影機_int': '攝影機'})
        final_df['datetime'] = pd.to_datetime(final_df['日期'] + ' ' + final_df['時間'])
        final_df = final_df.sort_values('datetime').drop(columns=['datetime'])
        final_df = final_df[['攝影機名稱', '車牌', '單位', '日期', '時間', '攝影機', '經度', '緯度']]
        
        output_filename = 'vehicle_behavior_dataset_2months_final.csv'
        final_df.to_csv(output_filename, index=False)
        
        print(f"\n成功生成兩個月的假資料！")
        print(f"總共生成了 {len(final_df)} 筆紀錄，包含 {final_df['車牌'].nunique()} 輛具有角色化車牌的車輛。")
        print(f"檔案已儲存為：{output_filename}")
        print("\n資料預覽：")
        print(final_df.head())
    else:
        print("錯誤：沒有生成任何資料。")

if __name__ == '__main__':
    generate_vehicle_behavior_dataset()

--- 開始生成資料集 (終極版 - 緊密跟隨 & 全員多據點) ---
1. 攝影機網路建立完成，共 29 支攝影機。
2. 車輛角色與生成邏輯定義完成。
3. 開始執行兩個月的每日劇本...
劇本執行完畢。
4. 開始格式化並儲存最終資料...

成功生成兩個月的假資料！
總共生成了 4866 筆紀錄，包含 11 輛具有角色化車牌的車輛。
檔案已儲存為：vehicle_behavior_dataset_2months_final.csv

資料預覽：
          攝影機名稱        車牌     單位          日期        時間         攝影機  \
14  楊梅區-A住宅區-巷口  NOR-1002  楊梅派出所  2025-08-01  08:26:15  2288784783   
0   楊梅區-A住宅區-巷口  NOR-1001  楊梅派出所  2025-08-01  08:32:38  2288784783   
15     楊梅區-交流道A  NOR-1002  楊梅派出所  2025-08-01  08:34:16  2329904567   
28  楊梅區-A住宅區-巷口  NOR-1004  楊梅派出所  2025-08-01  08:35:26  2288784783   
1      楊梅區-交流道A  NOR-1001  楊梅派出所  2025-08-01  08:41:15  2329904567   

            經度         緯度  
14  121.100000  24.800000  
0   121.100000  24.800000  
15  121.142857  24.842857  
28  121.100000  24.800000  
1   121.142857  24.842857  


In [4]:
# 測試的

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_vehicle_behavior_dataset():
    """
    生成一個為期兩個月，包含多種使用角色化車牌的複雜行為模式軌跡資料集。
    此版本實現了以精確秒數差為基礎的緊密跟隨行為，並為所有毒販團夥成員增加了不固定的停留點。
    """
    print("--- 開始生成資料集 (終極版 - 精確時間差跟隨) ---")

    # --- 步驟 1: 定義虛擬世界 (攝影機網路) ---
    camera_data = {
        '攝影機': [
            'A01', 'A02', 'A03', 'A04', 'A05', # 區域 A: 主要住宅區
            'B01', 'B02', 'B03', 'B04', 'B05', # 區域 B: 辦公/商業區
            'C01', 'C02', 'C03',              # 區域 C: 次要住宅區/安全屋
            'D01', 'D02', 'D03',              # 區域 D: 休閒娛樂區/過夜點
            'S01', 'S02',                     # 區域 S: 可疑停留點
            'P01', 'P02', 'P03', 'P04',        # 區域 P: 交易點
            'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07' # 交通幹道
        ],
        '攝影機名稱': [
            '楊梅區-A住宅區-巷口', '楊梅區-A幹道-路口1', '楊梅區-A幹道-路口2', '楊梅區-A超市前', '楊梅區-交流道A',
            '中壢區-B園區-入口', '中壢區-B園區-內部', '中壢區-B商業大道-1', '中壢區-B商業大道-2', '中壢區-交流道B',
            '平鎮區-C住宅區-巷口', '平鎮區-C大學路', '平鎮區-C車站',
            '龍潭區-D風景區-入口', '龍潭區-D湖畔道路', '龍潭區-D市區',
            '龜山區-S工業區-路口', '龜山區-S倉庫前',
            '八德區-P公園-角落', '大溪區-P停車場-後方', '觀音區-P快速道路下', '新屋區-P濱海涼亭',
            '交通幹道-AS段', '交通幹道-AB段', '交通幹道-BC段', '交通幹道-CD段', '交通幹道-AD段', '交通幹道-BS段', '交通幹道-CP段'
        ],
        '單位': (['楊梅派出所']*5 + ['中壢派出所']*5 + ['平鎮派出所']*3 + 
               ['龍潭派出所']*3 + ['龜山派出所']*2 + ['八德分局']*4 + ['交通大隊']*7)
    }
    total_cameras = len(camera_data['攝影機'])
    camera_data['緯度'] = np.linspace(24.8, 25.1, total_cameras)
    camera_data['經度'] = np.linspace(121.1, 121.4, total_cameras)
    cameras_df = pd.DataFrame(camera_data)
    print(f"1. 攝影機網路建立完成，共 {total_cameras} 支攝影機。")

    paths = {
        "A_to_B": ['A01', 'A05', 'T02', 'B05', 'B01', 'B02'], "B_to_A": ['B02', 'B01', 'B05', 'T02', 'A05', 'A01'],
        "A_to_S": ['A01', 'A05', 'T01', 'S01'], "S_to_B": ['S01', 'T06', 'B05', 'B01', 'B02'],
        "B_to_D": ['B02', 'B03', 'T03', 'C02', 'T04', 'D01'],
        "C_to_P": ['C01', 'T07', 'P01', 'P02'], "P_to_A": ['P02', 'P04', 'T05', 'A02', 'A01'],
        "P_to_C": ['P01','T07','C02','C01'], "P_to_D": ['P04','T04','D02','D03'],
        "B_internal_1": ['B01', 'B03', 'B04'], "B_internal_2": ['B04', 'B05', 'B01'],
    }

    # --- 步驟 2: 定義車輛角色與生成邏輯 ---
    plates = {
        "normal":     [f"NOR-{1001+i}" for i in range(5)],
        "dealer":     [f"DEL-8888"],
        "accomplice": [f"ACC-0001", "ACC-0002"],
        "uncertain":  [f"UNC-1001", "UNC-2001", "UNC-3001"],
        "sparse":     [f"SPA-7777", "SPA-8888"],
        "territorial":[f"TER-9999"]
    }
    all_sightings = []

    def generate_trip(plate, path, start_time):
        current_time = start_time
        for cam_id in path:
            delay = random.uniform(8, 12)
            current_time += timedelta(minutes=delay)
            cam_info = cameras_df[cameras_df['攝影機'] == cam_id].iloc[0]
            # 統一使用 datetime 物件儲存
            all_sightings.append({'車牌': plate, 'datetime': current_time, **cam_info.to_dict()})
        return current_time

    def generate_stay(plate, location_cam_id, start_time, duration_hours):
        end_time = start_time + timedelta(hours=duration_hours)
        cam_info = cameras_df[cameras_df['攝影機'] == location_cam_id].iloc[0]
        # 統一使用 datetime 物件儲存
        all_sightings.append({'車牌': plate, 'datetime': start_time, **cam_info.to_dict()})
        all_sightings.append({'車牌': plate, 'datetime': end_time, **cam_info.to_dict()})
        return end_time
    
    # --- NEW FUNCTION ---
    def generate_convoy_trip(dealer_plate, accomplice1_plate, accomplice2_plate, path, start_time):
        """
        生成一個緊密跟隨的車隊軌跡。
        主車(dealer)正常行駛，前車(accomplice1)提前5-15秒，後車(accomplice2)延後5-15秒。
        """
        dealer_sightings = []
        current_time = start_time
        
        # 1. 先生成主車(dealer)的完整軌跡
        for cam_id in path:
            delay = random.uniform(8, 12) # 主車的正常行駛延遲
            current_time += timedelta(minutes=delay)
            cam_info = cameras_df[cameras_df['攝影機'] == cam_id].iloc[0]
            dealer_sightings.append({'車牌': dealer_plate, 'datetime': current_time, **cam_info.to_dict()})

        # 2. 根據主車的軌跡，生成前後車的紀錄
        for sighting in dealer_sightings:
            dealer_time = sighting['datetime']
            
            # 生成前車紀錄 (時間提前)
            front_offset = timedelta(seconds=random.uniform(5, 15))
            front_sighting = sighting.copy()
            front_sighting['車牌'] = accomplice1_plate
            front_sighting['datetime'] = dealer_time - front_offset
            
            # 生成後車紀錄 (時間延後)
            rear_offset = timedelta(seconds=random.uniform(5, 15))
            rear_sighting = sighting.copy()
            rear_sighting['車牌'] = accomplice2_plate
            rear_sighting['datetime'] = dealer_time + rear_offset
            
            # 將三筆紀錄加入總列表
            all_sightings.append(front_sighting)
            all_sightings.append(sighting) # 主車紀錄
            all_sightings.append(rear_sighting)
            
        return dealer_sightings[-1]['datetime'] # 返回車隊最後的時間
    
    print("2. 車輛角色與生成邏輯定義完成。")

    # --- 步驟 3: 執行兩個月的「劇本」---
    print("3. 開始執行兩個月的每日劇本...")
    start_date = datetime(2025, 8, 1)
    end_date = datetime(2025, 9, 30)
    
    dealer_run_dates = [d.date() for d in pd.date_range(start_date, end_date) if random.random() < 0.4]
    accomplice_bases = ['A04', 'C03', 'D02']
    accomplice1_current_base = random.choice(accomplice_bases)
    accomplice2_current_base = random.choice(accomplice_bases)

    current_date = start_date
    while current_date <= end_date:
        day_of_week = current_date.weekday()

        # --- 正常上班族劇本 ---
        for plate in plates["normal"]:
            if day_of_week < 5 and random.random() < 0.95:
                start_morning = current_date.replace(hour=8, minute=random.randint(0, 45))
                end_morning = generate_trip(plate, paths['A_to_B'], start_morning)
                generate_stay(plate, 'B02', end_morning, duration_hours=random.uniform(8, 9))
                start_evening = end_morning + timedelta(hours=random.uniform(8,9.5))
                generate_trip(plate, paths['B_to_A'], start_evening)
        
        # --- 毒販與跟班協同劇本 (精確時間差跟隨 & 全員多據點) ---
        dealer_plate = plates["dealer"][0]
        accomplice1_plate, accomplice2_plate = plates["accomplice"]

        if current_date.date() in dealer_run_dates:
            start_run = current_date.replace(hour=22, minute=random.randint(0, 59))
            
            # --- MODIFIED SECTION ---
            # 透過加減秒數，直接模擬緊密跟隨的車隊
            current_time = generate_convoy_trip(
                dealer_plate=dealer_plate, 
                accomplice1_plate=accomplice1_plate,
                accomplice2_plate=accomplice2_plate,
                path=paths['C_to_P'], 
                start_time=start_run
            )
            
            # 主車在交易點停留後返回 A 區過夜點
            current_time += timedelta(minutes=random.randint(5,15))
            end_time = generate_trip(dealer_plate, paths['P_to_A'], current_time)
            generate_stay(dealer_plate, 'A01', end_time, random.uniform(6,10))
            
            # 跟班們各自返回一個隨機的、不固定的據點
            accomplice1_current_base = random.choice(accomplice_bases)
            accomplice2_current_base = random.choice(accomplice_bases)
            generate_stay(accomplice1_plate, accomplice1_current_base, start_run + timedelta(hours=2), random.uniform(7,10))
            generate_stay(accomplice2_plate, accomplice2_current_base, start_run + timedelta(hours=2.5), random.uniform(7,10))
        else:
            # 在沒有任務的日子裡，跟班們也會隨機移動並更換據點
            if random.random() < 0.5:
                start_move = current_date.replace(hour=random.randint(10,14))
                accomplice1_current_base = random.choice(accomplice_bases)
                end_move = generate_trip(accomplice1_plate, random.choice([paths['A_to_B'], paths['B_to_D']]), start_move)
                generate_stay(accomplice1_plate, accomplice1_current_base, end_move, random.uniform(8,12))
        
        # --- 地盤型車輛劇本 ---
        territorial_plate = plates["territorial"][0]
        if day_of_week < 5 and random.random() < 0.9:
            start_work = current_date.replace(hour=9, minute=random.randint(0, 20))
            current_time = generate_stay(territorial_plate, 'B01', start_work, random.uniform(0.2,0.5))
            for _ in range(random.randint(4, 8)):
                path_key = random.choice(["B_internal_1", "B_internal_2"])
                current_time = generate_trip(territorial_plate, paths[path_key], current_time)
                current_time = generate_stay(territorial_plate, random.choice(['B03', 'B04', 'B05']), current_time, random.uniform(0.1, 0.3))
        
        # --- 資料不足車輛 ---
        for plate in plates["sparse"]:
            if random.random() < 0.05:
                start_time = current_date.replace(hour=random.randint(9,18))
                generate_trip(plate, random.choice(list(paths.values())), start_time)

        current_date += timedelta(days=1)
    print("劇本執行完畢。")

    # --- 步驟 4: 格式化並儲存檔案 ---
    if all_sightings:
        print("4. 開始格式化並儲存最終資料...")
        final_df = pd.DataFrame(all_sightings)
        
        # 從 datetime 欄位拆分出日期和時間
        final_df['日期'] = final_df['datetime'].dt.strftime('%Y-%m-%d')
        final_df['時間'] = final_df['datetime'].dt.strftime('%H:%M:%S')

        final_df['攝影機_int'] = final_df['攝影機'].apply(lambda x: abs(hash(x)) % (10**9) + 2000000000)
        final_df = final_df.drop(columns=['攝影機'])
        final_df = final_df.rename(columns={'攝影機_int': '攝影機'})
        
        # 排序後再丟棄 datetime 輔助欄位
        final_df = final_df.sort_values('datetime').drop(columns=['datetime'])
        final_df = final_df[['攝影機名稱', '車牌', '單位', '日期', '時間', '攝影機', '經度', '緯度']]
        
        output_filename = 'vehicle_behavior_dataset_2months_precise_offset.csv'
        final_df.to_csv(output_filename, index=False)
        
        print(f"\n成功生成兩個月的假資料！")
        print(f"總共生成了 {len(final_df)} 筆紀錄，包含 {final_df['車牌'].nunique()} 輛具有角色化車牌的車輛。")
        print(f"檔案已儲存為：{output_filename}")
        print("\n資料預覽：")
        print(final_df.head())
    else:
        print("錯誤：沒有生成任何資料。")

if __name__ == '__main__':
    generate_vehicle_behavior_dataset()

--- 開始生成資料集 (終極版 - 精確時間差跟隨) ---
1. 攝影機網路建立完成，共 29 支攝影機。
2. 車輛角色與生成邏輯定義完成。
3. 開始執行兩個月的每日劇本...
劇本執行完畢。
4. 開始格式化並儲存最終資料...

成功生成兩個月的假資料！
總共生成了 4854 筆紀錄，包含 11 輛具有角色化車牌的車輛。
檔案已儲存為：vehicle_behavior_dataset_2months_precise_offset.csv

資料預覽：
          攝影機名稱        車牌     單位          日期        時間         攝影機  \
0   楊梅區-A住宅區-巷口  NOR-1001  楊梅派出所  2025-08-01  08:10:22  2874031857   
56  楊梅區-A住宅區-巷口  NOR-1005  楊梅派出所  2025-08-01  08:16:08  2874031857   
1      楊梅區-交流道A  NOR-1001  楊梅派出所  2025-08-01  08:19:13  2215275639   
57     楊梅區-交流道A  NOR-1005  楊梅派出所  2025-08-01  08:25:30  2215275639   
2      交通幹道-AB段  NOR-1001   交通大隊  2025-08-01  08:29:58  2252769213   

            經度         緯度  
0   121.100000  24.800000  
56  121.100000  24.800000  
1   121.142857  24.842857  
57  121.142857  24.842857  
2   121.346429  25.046429  
